### Minimum Number of Flips to Convert Binary Matrix to Zero Matrix
Given a m x n binary matrix mat. In one step, you can choose one cell and flip it and all the four neighbours of it if they exist (Flip is changing 1 to 0 and 0 to 1). A pair of cells are called neighboors if they share one edge.

Return the minimum number of steps required to convert mat to a zero matrix or -1 if you cannot.

Binary matrix is a matrix with all cells equal to 0 or 1 only.

Zero matrix is a matrix with all cells equal to 0.

In [1]:
class Solution:
    def minFlips(self, mat) -> int:
        hashcode = 0; row = len(mat); col = len(mat[0])
        for i in range(row):
            for j in range(col):
                if mat[i][j] == 1:
                    k = i*col + j
                    hashcode |= 1<<k
                    
        if hashcode == 0:
            return 0
        
        queue = [hashcode]; seen = {hashcode}
        level = 0
        
        while queue:
            size = len(queue)
            for _ in range(size):
                hashcode = queue.pop(0)
                if hashcode == 0:
                    return level
                for i in range(row):
                    for j in range(col):
                        new_hashcode = self.switch(i, j, row, col, hashcode)
                        if new_hashcode not in seen:
                            seen.add(new_hashcode)
                            queue.append(new_hashcode)
            level += 1
        return -1
        
    def switch(self, i, j, r, c, hashcode):
        for x, y in [(i,j), (i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            k = x*c + y
            if 0<=x<r and 0<=y<c:
                hashcode ^= 1<<k
        
        return hashcode

grid = [[1,1,1],[1,0,1],[0,0,0]]
Solution().minFlips(grid)

6

###  Shortest Path Visiting All Nodes
An undirected, connected graph of N nodes (labeled 0, 1, 2, ..., N-1) is given as graph.

graph.length = N, and j != i is in the list graph[i] exactly once, if and only if nodes i and j are connected.

Return the length of the shortest path that visits every node. You may start and stop at any node, you may revisit nodes multiple times, and you may reuse edges.

In [2]:
def shortestPathLength(graph) -> int:
    queue = []; seen = set()
    num_nodes = len(graph)
    for i in range(num_nodes):
        queue.append((i, 1<<i))
        seen.add((i, 1<<i))

    level = 0
    while queue:
        size = len(queue)
        for _ in range(size):
            node, hashcode = queue.pop(0)
            if hashcode == 2**num_nodes-1:
                return level
            for i in graph[node]:
                if (i, hashcode | 1<<i) not in seen:
                    seen.add((i, hashcode|1<<i))
                    queue.append((i, hashcode|1<<i))
        level += 1
    
graph = [[1],[0,2,4],[1,3,4],[2],[1,2]]
shortestPathLength(graph)

4

### Bus Routes
We have a list of bus routes. Each routes[i] is a bus route that the i-th bus repeats forever. For example if routes[0] = [1, 5, 7], this means that the first bus (0-th indexed) travels in the sequence 1->5->7->1->5->7->1->... forever.

We start at bus stop S (initially not on a bus), and we want to go to bus stop T. Travelling by buses only, what is the least number of buses we must take to reach our destination? Return -1 if it is not possible.

In [10]:
from collections import defaultdict
class Solution:
    def numBusesToDestination(self, routes, S: int, T: int) -> int:
        if S == T: return 0
        hashmap = defaultdict(set)
        queue = []; seen = set()
        for route_index, route in enumerate(routes):
            for stop in route:
                hashmap[stop].add(route_index)
        
        for route_index in hashmap[S]:
            queue.append(route_index)
            seen.add(route_index)
        
        num_buses = 1
        while queue:
            size = len(queue)
            for _ in range(size):
                route_index = queue.pop(0)
                if route_index in hashmap[T]:
                    return num_buses
                for stop in routes[route_index]:
                    for next_index in hashmap[stop]:
                        if next_index not in seen:
                            seen.add(next_index)
                            queue.append(next_index)
            num_buses += 1
        return -1

routes = [[1, 2, 7], [3, 6, 7]]
S = 1
T = 6
Solution().numBusesToDestination(routes, S, T)

2

### Sliding Puzzle
On a 2x3 board, there are 5 tiles represented by the integers 1 through 5, and an empty square represented by 0.

A move consists of choosing 0 and a 4-directionally adjacent number and swapping it.

The state of the board is solved if and only if the board is [[1,2,3],[4,5,0]].

Given a puzzle board, return the least number of moves required so that the state of the board is solved. If it is impossible for the state of the board to be solved, return -1.

In [8]:
class Solution:
    def slidingPuzzle(self, board) -> int:
        target = [1,2,3,4,5,0]
        flat_board = []
        for i in range(len(board)):
            for j in range(len(board[0])):
                flat_board.append(board[i][j])
                if board[i][j] == 0:
                    zero_pos = (i,j)
        
        queue = [(flat_board, zero_pos)]
        seen = {tuple(flat_board)}
        level = 0
        
        while queue:
            size = len(queue)
            for _ in range(size):
                flat_board, zero_pos = queue.pop(0)
                if flat_board == target:
                    return level
                i, j = zero_pos
                for x,y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
                    if 0<=x<len(board) and 0<=y<len(board[0]):
                        new_flat_board = self.swap(flat_board[:], i, j, x, y, board)
                        if tuple(new_flat_board) not in seen:
                            seen.add(tuple(new_flat_board))
                            queue.append((new_flat_board, (x,y)))
                
            level += 1
        return -1
    
    def swap(self, flat_board, i, j, x, y, board):
        p = i*len(board[0]) + j
        q = x*len(board[0]) + y
        flat_board[p], flat_board[q] = flat_board[q], flat_board[p]
        return flat_board

board = [[4,1,2],[5,0,3]]   
obj = Solution()
obj.slidingPuzzle(board)

5

### Find Eventual Safe States
In a directed graph, we start at some node and every turn, walk along a directed edge of the graph.  If we reach a node that is terminal (that is, it has no outgoing directed edges), we stop.

Now, say our starting node is eventually safe if and only if we must eventually walk to a terminal node.  More specifically, there exists a natural number K so that for any choice of where to walk, we must have stopped at a terminal node in less than K steps.

Which nodes are eventually safe?  Return them as an array in sorted order.

The directed graph has N nodes with labels 0, 1, ..., N-1, where N is the length of graph.  The graph is given in the following form: graph[i] is a list of labels j such that (i, j) is a directed edge of the graph.

In [4]:
class Solution:
    def eventualSafeNodes(self, graph):
        self.hashmap = {}
        ans = []
        for i in range(len(graph)):
            if i in self.hashmap and self.hashmap[i] == 'safe':
                ans.append(i)
            if i not in self.hashmap:
                if not self.is_unsafe(i, graph):
                    ans.append(i)
        return ans
    
    def is_unsafe(self, node, graph):
        self.hashmap[node] = 'unsafe'
        for neigh in graph[node]:
            if neigh in self.hashmap and self.hashmap[neigh] == 'unsafe':
                return True
            elif neigh not in self.hashmap and self.is_unsafe(neigh, graph):
                return True
        
        self.hashmap[node] = 'safe'
        return False

graph = [[1,2],[2,3],[5],[0],[5],[],[]]
obj = Solution()
obj.eventualSafeNodes(graph)

[2, 4, 5, 6]

### Bricks Falling When Hit
We have a grid of 1s and 0s; the 1s in a cell represent bricks.  A brick will not drop if and only if it is directly connected to the top of the grid, or at least one of its (4-way) adjacent bricks will not drop.

We will do some erasures sequentially. Each time we want to do the erasure at the location (i, j), the brick (if it exists) on that location will disappear, and then some other bricks may drop because of that erasure.

Return an array representing the number of bricks that will drop after each erasure in sequence.

In [2]:
class Solution:
    def hitBricks(self, grid, hits):
        for i, j in hits:
            grid[i][j] = 0 if grid[i][j] == 1 else -1
        
        for i in range(len(grid[0])):
            if grid[0][i] == 1:
                self.dfs(0, i, grid)
        
        ans = [0]*len(hits)
        
        for i in range(len(hits)-1, -1, -1):
            x, y = hits[i]
            grid[x][y] += 1
            if grid[x][y] == 1 and self.is_connected(x, y, grid):
                size = self.dfs(x, y, grid) - 1
                ans[i] = size
                
        return ans
    
    def is_connected(self, i, j, grid):
        if i == 0:
            return True
        for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            if 0<=x<len(grid) and 0<=y<len(grid[0]) and grid[x][y] == 2:
                return True
        return False
    
    def dfs(self, i, j, grid):
        grid[i][j] = 2
        ans = 1
        for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            if 0<=x<len(grid) and 0<=y<len(grid[0]) and grid[x][y] == 1:
                ans += self.dfs(x, y, grid)
        return ans
                
obj = Solution()
grid = [[1,0,0,0],[1,1,0,0]]
hits = [[1,1],[1,0]]
obj.hitBricks(grid, hits)

[0, 0]

### Race Car
Your car starts at position 0 and speed +1 on an infinite number line.  (Your car can go into negative positions.)

Your car drives automatically according to a sequence of instructions A (accelerate) and R (reverse).

When you get an instruction "A", your car does the following: position += speed, speed *= 2.

When you get an instruction "R", your car does the following: if your speed is positive then speed = -1 , otherwise speed = 1.  (Your position stays the same.)

For example, after commands "AAR", your car goes to positions 0->1->3->3, and your speed goes to 1->2->4->-1.

Now for some target position, say the length of the shortest sequence of instructions to get there.

In [13]:
import collections
def racecar(target: int) -> int:
    queue = collections.deque(); seen = {(0, 1)}
    queue.append((0,1))
    steps = 0
    while queue:
        size = len(queue)
        for _ in range(size):
            pos, speed = queue.popleft()
            if pos == target:
                return steps

            # A
            new_pos = pos + speed
            new_speed = speed*2
            if (new_pos, new_speed) not in seen:
                seen.add((new_pos, new_speed))
                queue.append((new_pos, new_speed))

            # R
            new_speed = -1 if speed > 0 else 1
            if (pos, new_speed) not in seen:
                seen.add((pos, new_speed))
                queue.append((pos, new_speed))
        steps += 1

racecar(6)

5

### Most Stones Removed with Same Row or Column
On a 2D plane, we place stones at some integer coordinate points.  Each coordinate point may have at most one stone.

Now, a move consists of removing a stone that shares a column or row with another stone on the grid.

What is the largest possible number of moves we can make?

In [7]:
from collections import defaultdict
class Solution:
    def removeStones(self, stones) -> int:
        self.dic_row = defaultdict(list)
        self.dic_col = defaultdict(list)
        for x, y in stones:
            self.dic_row[x].append((x,y))
            self.dic_col[y].append((x,y))
        
        seen = set(); ans = 0
        for x, y in stones:
            if (x,y) not in seen:
                size = self.get_size(x, y, seen)
                ans += size-1
        return ans
    
    def get_size(self, i, j, seen):
        seen.add((i,j))
        size = 1
        
        for x, y in self.dic_row[i]:
            if (x,y) not in seen:
                size += self.get_size(x, y, seen)
        
        for x, y in self.dic_col[j]:
            if (x,y) not in seen:
                size += self.get_size(x, y, seen)
        
        return size

stones = [[0,0],[0,1],[1,0],[1,2],[2,1],[2,2]]
Solution().removeStones(stones)

5